> > **Introdução**: Neste dataset, temos várias features que podem ser importantes para previsão do preço de casas.

> > **Objetivo:** inicialmente, é necessário fazer algumas manipulações de modo que precisamos escolher o melhor modelo para o cliente aplicar em seus dados, seja com dashboard, em alguma apresentação, ou paínel. Dessa forma, é necessário fazer previsões com base em alguns parâmetros ('LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd') do conjunto de dados.

# Vamos analizar o RandomForest[](http://)

### 1 Importação dos dados e Limpeza (um pouco de EDA)

In [1]:
# Importando as bibliotecas necessárias para manipulação dos dados
import numpy as np
import pandas as pd

In [2]:
# Definindo e importando os dados para visualização
df1 = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df2 = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [3]:
# Visualizando os conjunto dados treino
df2.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
# Visualizando algumas informações baísicas, como quantidade de linhas, média
# std, quartis/percentiles
df2.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [5]:
# Verificando os nulos (está aparecendo zero porque na etapa posterior eu fiz
#a manipulação utilizando um replace em LotFrontage substituindo os nulos pela média)
df2.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [6]:
# verificando a quantidade de linhas e colunas
df2.shape

(1460, 81)

In [7]:
# analizando o type de dado
type(df2['LotFrontage'])

pandas.core.series.Series

In [8]:
# fazendo a substituição
df2['LotFrontage'].fillna(np.mean(df2['LotFrontage']), inplace = True)

### 2 Início do Algoritmo de ML

In [9]:
# importando os pacotes necessários iniciar nosso algoritmo.
# Para esse caso, vou utlizar o RandomForestRegressor, mas também utilizarei o
# DecisionTree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [10]:
# Variável que eu quero prever
y = df2.SalePrice

In [11]:
# Colunas que irei analisar, geralmente são dados tipo int ou float
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

In [12]:
# Para ficar mais fácil na hora de chamar, vamos declarar uma variável
X = df2[features]

In [13]:
# Verificando
X.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
0,8450,2003,856,854,2,3,8
1,9600,1976,1262,0,2,3,6
2,11250,2001,920,866,2,3,6
3,9550,1915,961,756,1,3,7
4,14260,2000,1145,1053,2,4,9


In [14]:
# Utilizando o split, e separando os dados treino em 30%, para falar a verdade
# suponho que seja 30% de 30% se os dados já estiverem separados
# Ex: Temos 1000 linhas e separamos para 70% em treino e 30% em teste
# Como utilizei test_size = 0.3, suponho que eu esteja treinando 0,3 * 700 = 210 linhas
# Mas, seguindo...
train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.3, random_state=1)

In [15]:
# Precisamos passar os parametros para função, cuidado com o overfitting e underfitting
model1 = RandomForestRegressor(n_estimators = 150,min_samples_leaf=8,random_state = 0, n_jobs = -1)

In [16]:
# Treinando o modelo com o fit
model1.fit(train_X,train_y)

RandomForestRegressor(min_samples_leaf=8, n_estimators=150, n_jobs=-1,
                      random_state=0)

In [17]:
# Fazendo as previsões com o conjunto de validação (não é o dado teste)
model1_val_predictions = model1.predict(val_X)

In [18]:
model1_val_mse= np.sqrt(mean_squared_error(model1_val_predictions, val_y))

In [19]:
# Verificando o erro:
model1_val_mae = mean_absolute_error(model1_val_predictions, val_y)

In [20]:
print("Validation MSE for Random Forest Model: {:,.0f}".format(model1_val_mse))

Validation MSE for Random Forest Model: 38,020


In [21]:
# Qual erro final, para este modelo, com o conjunto de dados teste
print("Validation MAE for Random Forest Model: {:,.0f}".format(model1_val_mae))

Validation MAE for Random Forest Model: 24,285


In [22]:
# Agora, vamos utilizar os dados para treinar os modelos com os dados que precisamos prever
model1_on_full_data = RandomForestRegressor(random_state = 1)

In [23]:
# Treinando
model1_on_full_data.fit(X,y)

RandomForestRegressor(random_state=1)

In [24]:
# Analizando as colunas, do nosso conjunto dados teste!!
test_X = df1[features]

In [25]:
# Quais previsões?
test_preds = model1_on_full_data.predict(test_X)

In [26]:
# Printaremos
test_preds[:10]

array([122656.58, 156789.  , 182959.  , 178102.  , 189049.48, 180979.45,
       172797.  , 173716.72, 187535.19, 116171.5 ])

In [27]:
# Criaremos um dataframe com o id e saleprice, conforme foi pedido
output = pd.DataFrame({'Id': df1.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)

In [28]:
# Resultados da previsão com o modelo de RandomForest
output.head()

,Id,SalePrice
0,1461,122656.58
1,1462,156789.00
2,1463,182959.00
3,1464,178102.00
4,1465,189049.48


# Vamos analizar agora com o DecisionTreeRegressor[](http://)

In [29]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [30]:
model2 = DecisionTreeRegressor(min_samples_leaf=21, random_state=0)
model2.fit(train_X,train_y)

DecisionTreeRegressor(min_samples_leaf=21, random_state=0)

In [31]:
model2_val_predictions = model2.predict(val_X)

In [32]:
model2_val_mae = mean_absolute_error(model2_val_predictions, val_y)

In [33]:
print("Validation MAE for Decision Tree Model: {:,.0f}".format(model2_val_mae))

Validation MAE for Decision Tree Model: 29,444


In [34]:
model2_val_mse= np.sqrt(mean_squared_error(model2_val_predictions, val_y))

In [35]:
print("Validation MSE for Decision Tree Model: {:,.0f}".format(model2_val_mse))

Validation MSE for Decision Tree Model: 44,802


In [36]:
model2_on_full_data = DecisionTreeRegressor(random_state = 1)

In [37]:
model2_on_full_data.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [38]:
test_X1 = df1[features]

In [39]:
test_preds = model2_on_full_data.predict(test_X1)

In [40]:
# Printaremos
test_preds[:10]

array([ 94750., 137500., 173000., 178000., 189000., 165400., 173000.,
       163990., 180000.,  97000.])

In [41]:
output1 = pd.DataFrame({'Id': df1.Id,
                       'SalePrice': test_preds})
output1.to_csv('submission1.csv', index=False)

In [42]:
output1.head()

,Id,SalePrice
0,1461,94750.0
1,1462,137500.0
2,1463,173000.0
3,1464,178000.0
4,1465,189000.0


In [43]:
Resultados = pd.DataFrame({'RandomForest':["model1_val_mse: {:,.0f}".format(model1_val_mse),"model1_val_mae: {:,.0f}".format(model1_val_mae)],
        'DecisionTree': ["model2_val_mse: {:,.0f}".format(model2_val_mse),"model2_val_msa: {:,.0f}".format(model2_val_mae)]})

#print("Validation MAE for Random Forest Model: {:,.0f}".format(model1_val_mae))
#model1_val_mse
#model1_val_msa
#model2_val_mse
#model2_val_msa

### 3 Resultados

In [44]:
print(Resultados)

             RandomForest            DecisionTree
0  model1_val_mse: 38,020  model2_val_mse: 44,802
1  model1_val_mae: 24,285  model2_val_msa: 29,444


# **Sendo assim, ficaremos com o model1, onde temos um mse e mae baixo, em relação ao model2.**

# Ou seja, nesse caso, RandomForest obteve mais sucesso!!